In [1]:
import pandas as pd

In [2]:
url = 'https://www.juntadeandalucia.es/institutodeestadisticaycartografia/badea/stpivot/stpivot/Print?cube=013d884c-cfc5-4c21-9e78-e26225682afd&type=3&foto=si&ejecutaDesde=&codConsulta=38668&consTipoVisua=JP'
fields = ['Lugar de residencia', 'Medida', 'Valor']
df = pd.read_csv(url, delimiter=';', usecols=fields, decimal=',')

In [3]:
df['Valor'] = pd.to_numeric(df['Valor'],errors='coerce')

In [4]:
dfAnterior = pd.read_csv('covid_huelva.csv', index_col='Lugar de residencia')

In [5]:
# Confirmados PCR provincia Huelva
p = df[df['Lugar de residencia'] == 'Huelva'][df['Medida'] == 'Confirmados PCR']['Valor'].values[0]
print('Confirmados PCR en la provincia de Huelva: ', p)

Confirmados PCR en la provincia de Huelva:  1401.0


In [6]:
# Confirmados PCR Huelva capital
c = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PCR']['Valor'].values[0]
print('Confirmados PCR en Huelva capital: ', c)

Confirmados PCR en Huelva capital:  351.0


In [7]:
c7 = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PCR 7 días']['Valor'].values[0]
c14 = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PCR 14 días']['Valor'].values[0]
p = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Población']['Valor'].values[0]

In [8]:
print('Huelva (capital)\n')
#print('Confirmados PCR 7 días x 1,000 habitantes: ', f"{c7/(p/1000):.5f}")
print('Un positivo PCR cada', f"{p/c7:.0f}", 'personas en los últimos 7 días', '\n')
#print('Confirmados PCR 14 días x 1,000 habitantes: ', f"{c14/(p/1000):.5f}")
print('Un positivo PCR cada', f"{p/c14:.0f}", 'personas en los últimos 14 días')

Huelva (capital)

Un positivo PCR cada 3341 personas en los últimos 7 días 

Un positivo PCR cada 1545 personas en los últimos 14 días


In [9]:
filtro = ((df['Medida'] == 'Confirmados PCR')| (df['Medida'] == 'Confirmados PCR 7 días') | (df['Medida'] == 'Población' ) | (df['Medida'] == 'Confirmados PCR 14 días')) & (df['Valor'] > 0)
tabla = pd.pivot_table(df.loc[filtro], values='Valor', index='Lugar de residencia', columns='Medida').dropna()

In [10]:
# Reorder columns
tabla = tabla[['Confirmados PCR', 'Confirmados PCR 7 días', 'Confirmados PCR 14 días', 'Población']]

In [11]:
tabla['tasa7'] = tabla['Confirmados PCR 7 días']/tabla['Población']*100000
tabla['tasa14'] = tabla['Confirmados PCR 14 días']/tabla['Población']*100000

In [12]:
tabla['Diferencia PCR'] = tabla['Confirmados PCR'] - dfAnterior['Confirmados PCR']

In [13]:
# Nuevos casos desde la última captura
tabla[tabla['Diferencia PCR'] != 0].sort_values('Diferencia PCR', ascending=False)

Medida,Confirmados PCR,Confirmados PCR 7 días,Confirmados PCR 14 días,Población,tasa7,tasa14,Diferencia PCR
Lugar de residencia,,,,,,,
Huelva,1401.0,236.0,407.0,510743.0,46.207192,79.687827,22.0
Condado-Campiña,446.0,94.0,145.0,155057.0,60.622868,93.513998,14.0
Huelva-Costa,725.0,103.0,200.0,288115.0,35.749614,69.416726,7.0
Bollullos Par del Condado,83.0,37.0,49.0,14272.0,259.248879,343.329596,5.0
Huelva (capital),351.0,43.0,93.0,143663.0,29.931158,64.734831,4.0
Palma del Condado (La),54.0,18.0,23.0,10761.0,167.270700,213.734783,4.0
Manzanilla,13.0,9.0,9.0,2135.0,421.545667,421.545667,2.0
Sierra de Huelva-Andévalo Central,206.0,37.0,56.0,67571.0,54.757218,82.875790,2.0
Villablanca,5.0,3.0,3.0,2848.0,105.337079,105.337079,2.0


In [14]:
if tabla[tabla['Diferencia PCR'] != 0]['Diferencia PCR'].sum() > 0:
  # Save to csv
  tabla.to_csv('covid_huelva.csv', encoding='utf-8', header=True, index=True)  

In [15]:
tabla.sort_values('tasa14', ascending=False)

Medida,Confirmados PCR,Confirmados PCR 7 días,Confirmados PCR 14 días,Población,tasa7,tasa14,Diferencia PCR
Lugar de residencia,,,,,,,
Campillo (El),26.0,24.0,24.0,2023.0,1186.356896,1186.356896,1.0
Manzanilla,13.0,9.0,9.0,2135.0,421.545667,421.545667,2.0
Almendro (El),4.0,3.0,3.0,826.0,363.196126,363.196126,0.0
Bollullos Par del Condado,83.0,37.0,49.0,14272.0,259.248879,343.329596,5.0
Jabugo,5.0,1.0,5.0,2250.0,44.444444,222.222222,0.0
Palma del Condado (La),54.0,18.0,23.0,10761.0,167.270700,213.734783,4.0
Granado (El),2.0,1.0,1.0,516.0,193.798450,193.798450,0.0
Nerva,23.0,3.0,9.0,5235.0,57.306590,171.919771,0.0
Aroche,9.0,1.0,5.0,3073.0,32.541490,162.707452,0.0
